In [215]:
from bs4 import BeautifulSoup
import pandas as pd
import pickle
import requests
import time
from collections import Counter

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver
driver = webdriver.Chrome(chromedriver)

In [170]:
import re
import unidecode

In [3]:
home_url = 'http://millercenter.org/president/speeches'

In [4]:
page = requests.get(home_url)

In [5]:
soup = BeautifulSoup(page.text, "lxml")

In [6]:
links = soup.find_all("a")

In [7]:
links = soup.findAll(attrs={'class':'field-content'})

In [8]:
for link in links:
    if "icon" not in link.text:
        print(link.text)

February 28, 2017: Address to Joint Session of Congress
January 20, 2017: Inaugural Address
May 15, 2016: Commencement Address at Rutgers University
March 22, 2016: Remarks to the People of Cuba
January 12, 2016: 2016 State of the Union Address
June 26, 2015: Remarks in Eulogy for the Honorable Reverend Clementa Pickney
March 7, 2015: Remarks at the 50th Anniversary of the Selma Marches
January 20, 2015: 2015 State of the Union Address
November 20, 2014: Address to the Nation on Immigration
January 28, 2014: 2014 State of the Union Address
December 4, 2013: Speech on Economic Mobility
September 10, 2013: Address to the Nation on Syria


In [9]:
driver.get(home_url)

x = driver.find_elements_by_tag_name('a')
speeches = list()
for link in x:
    if not pd.isnull(link.get_attribute("href")) and "/presidential-speeches/" in link.get_attribute("href"):
        speeches.append(link.get_attribute("href"))

In [10]:
speech = speeches[0]
response = requests.get(speech)
print(speech)
a = BeautifulSoup(response.text, "lxml")
b = a.findAll(attrs={'class':'transcript-inner'})
len(b)

https://millercenter.org/the-presidency/presidential-speeches/february-28-2017-address-joint-session-congress


1

In [62]:
obama_url = "https://millercenter.org/president/obama"
obama_response = requests.get(obama_url)
obama_soup = BeautifulSoup(obama_response.text, "lxml")
facts = obama_soup.find_all(attrs={'class':'fast-facts-wrapper'})
name = obama_soup.find(attrs={'class': "fullname-data"})
name = name.text.split("\n")[2]
dates = obama_soup.find_all(attrs={"class": "date-data"})
dates = [(i.text.strip()) for i in dates]
info = obama_soup.find_all(attrs={"class": "string-data"})
info = [(i.text.strip()) for i in info]
number = obama_soup.find(attrs={"class": "number-data"})
number = int(number.text.split("\n")[3])

In [37]:
potus_url = "https://millercenter.org/president"
potus_response = requests.get(potus_url)
potus_soup = BeautifulSoup(potus_response.text, "lxml")
presidents = list()
x = potus_soup.find_all("a")
for link in x:
    if "href" in link.attrs:
        href = link.attrs["href"]
        if "/president/" in href:
            presidents.append(href)
presidents = list(set(presidents))

In [38]:
presidents

['/president/vanburen',
 '/president/arthur',
 '/president/washington',
 '/president/adams',
 '/president/taft',
 '/president/lbjohnson',
 '/president/trump',
 '/president/ford',
 '/president/clinton',
 '/president/fdroosevelt',
 '/president/lincoln',
 '/president/garfield',
 '/president/hayes',
 '/president/hoover',
 '/president/mckinley',
 '/president/coolidge',
 '/president/pierce',
 '/president/nixon',
 '/president/bharrison',
 '/president/bush',
 '/president/grant',
 '/president/taylor',
 '/president/fillmore',
 '/president/tyler',
 '/president/roosevelt',
 '/president/jefferson',
 '/president/truman',
 '/president/kennedy',
 '/president/wilson',
 '/president/gwbush',
 '/president/eisenhower',
 '/president/jackson',
 '/president/buchanan',
 '/president/monroe',
 '/president/cleveland',
 '/president/polk',
 '/president/reagan',
 '/president/obama',
 '/president/johnson',
 '/president/madison',
 '/president/carter',
 '/president/jqadams',
 '/president/harrison',
 '/president/harding

In [102]:
platform_url = "http://www.presidency.ucsb.edu/ws/index.php?pid=25849"
platform_response = requests.get(platform_url)
platform_soup = BeautifulSoup(platform_response.text, "lxml")
title = platform_soup.find(attrs={"class": "paperstitle"}).text
headlines = platform_soup.find_all("h3")
topics = list()
for a in headlines:
    topics.append(a.text)
topics = ", ".join(topics)
platform = list()
paras = platform_soup.find_all("p")
for p in paras:
    platform.append(p.text)
platform = " ".join(platform)
year = int(title.split(" ")[0])
party = title.split(" ")[1:3]
party = " ".join(party)
row = [{"Year": year, "Party": party, "Topics": topics, "Platform":platform}]
temp_df = pd.DataFrame(row)


In [107]:
temp_df

,Party,Platform,Topics,Year
0,Republican Party,We meet at a remarkable time in the life of ou...,"Preamble, Old Truths For The New Economy, The ...",2000


In [126]:
urls = ["http://www.presidency.ucsb.edu/ws/index.php?pid=25849", "http://www.presidency.ucsb.edu/ws/index.php?pid=29612"]
platforms = "http://www.presidency.ucsb.edu/platforms.php"
response = requests.get(platforms)
soup = BeautifulSoup(response.text, "lxml")
arr = soup.find_all("a")
links = []
for div in arr:
    if "href" in div.attrs:
        if "?pid=" in div.attrs["href"]:
            links.append(div.attrs["href"])


In [160]:
platform_first = list()
platform_last = list()
for url in links:
    platform_response = requests.get(url)
    platform_soup = BeautifulSoup(platform_response.text, "lxml")
    title = platform_soup.find(attrs={"class": "paperstitle"}).text
    if title.split(" ")[-1] == "Platform":
        platform_last.append(url)
    elif title.split(" ")[-1] != "Party":
        platform_first.append(url)
platform_first

['http://www.presidency.ucsb.edu/ws/index.php?pid=25848',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25847',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25846',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25845',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25844',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25843',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=29615',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25842',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=29570',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25841',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25840',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25839',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25838',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25837',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25836',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=25835',
 'http://www.presidency.ucsb.edu/ws/index.php?pid=29640',
 'http://www.p

In [164]:
rows = list()

In [165]:
for url in platform_last:
    platform_response = requests.get(url)
    platform_soup = BeautifulSoup(platform_response.text, "lxml")
    title = platform_soup.find(attrs={"class": "paperstitle"}).text
    print(title)
    headlines = platform_soup.find_all("h3")
    topics = list()
    for a in headlines:
        topics.append(a.text)
    headlines = platform_soup.find_all("h4")
    for a in headlines:
        topics.append(a.text)
    headlines = platform_soup.find_all("b")
    for a in headlines:
        if "1. " not in a.text and "2. " not in a.text:
            topics.append(a.text)
    topics = ", ".join(topics)
    platform = list()
    paras = platform_soup.find_all("p")
    for p in paras:
        platform.append(p.text)
    platform = " ".join(platform)
    year = int(title.split(" ")[0])
    party = title.split(" ")[1]
    #row = [{"Year": year, "Party": party, "Topics": topics, "Platform":platform}]
    row = [year, party, topics, platform]
    rows.append(row)

2016 Democratic Party Platform
2016 Republican Party Platform
2012 Democratic Party Platform
2012 Republican Party Platform
2008 Democratic Party Platform
2008 Republican Party Platform
2004 Democratic Party Platform
2004 Republican Party Platform
2000 Democratic Party Platform
2000 Republican Party Platform
1996 Democratic Party Platform
1992 Democratic Party Platform
1988 Democratic Party Platform
1984 Democratic Party Platform
1980 Democratic Party Platform
1976 Democratic Party Platform
1972 Democratic Party Platform
1968 Democratic Party Platform
1964 Democratic Party Platform
1960 Democratic Party Platform
1956 Democratic Party Platform
1952 Democratic Party Platform
1948 Democratic Party Platform
1944 Democratic Party Platform
1940 Democratic Party Platform
1936 Democratic Party Platform
1932 Democratic Party Platform
1928 Democratic Party Platform
1924 Democratic Party Platform
1920 Democratic Party Platform
1916 Democratic Party Platform
1912 Democratic Party Platform
1908 Dem

In [166]:
for url in platform_first:
    platform_response = requests.get(url)
    platform_soup = BeautifulSoup(platform_response.text, "lxml")
    title = platform_soup.find(attrs={"class": "paperstitle"}).text
    print(title)
    headlines = platform_soup.find_all("h3")
    topics = list()
    for a in headlines:
        topics.append(a.text)
    headlines = platform_soup.find_all("h4")
    for a in headlines:
        topics.append(a.text)
    headlines = platform_soup.find_all("b")
    for a in headlines:
        if "1. " not in a.text and "2. " not in a.text:
            topics.append(a.text)
    topics = ", ".join(topics)
    platform = list()
    paras = platform_soup.find_all("p")
    for p in paras:
        platform.append(p.text)
    platform = " ".join(platform)
    year = int(title.split(" ")[-1])
    party = title.split(" ")[0]
    #row = [{"Year": year, "Party": party, "Topics": topics, "Platform":platform}]
    row = [year, party, topics, platform]
    rows.append(row)

Republican Party Platform of 1996
Republican Party Platform of 1992
Republican Party Platform of 1988
Republican Party Platform of 1984
Republican Party Platform of 1980
Republican Party Platform of 1976
Libertarian Party Platform of 1972
Republican Party Platform of 1972
American Independent Party Platform of 1968
Republican Party Platform of 1968
Republican Party Platform of 1964
Republican Party Platform of 1960
Republican Party Platform of 1956
Republican Party Platform of 1952
Republican Party Platform of 1948
Republican Party Platform of 1944
Republican Party Platform of 1940
Republican Party Platform of 1936
Republican Party Platform of 1932
Republican Party Platform of 1928
Progressive Party Platform of 1924
Republican Party Platform of 1924
Republican Party Platform of 1920
Republican Party Platform of 1916
Progressive Party Platform of 1912
Republican Party Platform of 1912
Republican Party Platform of 1908
Republican Party Platform of 1904
Republican Party Platform of 1900
R

In [169]:
platform_df = pd.DataFrame(rows, columns=["Year", "Party", "Topics", "Platform"])

In [172]:
platform_df = platform_df.sort_values("Year", ascending=False).reset_index()

In [228]:
url = "http://www.presidency.ucsb.edu/nomination.php"
driver.get(url)
a = driver.find_elements_by_xpath("//tr")
for item in a[37:]:
    print(item.text)


• 2001 Transition
• White House Media Pool Reports
Data Archive
Data Index
Media Archive
Audio/Video Index
Elections
Election Index
Florida 2000
Links
Presidential Libraries
• Interesting Convention Facts
• Ballots Required to Nominate Candidates at National Conventions
• Cities That Have Hosted National Conventions
• Political Party Platforms
• Nomination Acceptance Speeches
• Days Before or After the First Day of Convention Vice-Presidential Nominees Were Selected

Presidential Nomination
Acceptance Speeches and Letters
1880 - 2016

compiled by Gerhard Peters

     Democratic Party
Nominee Date Location Length
did nominee win state hosting convention?
in time in words
Hillary Clinton July 28, 2016 Philadelphia, PA 56:34 5389 TBD
Barack Obama September 6, 2012 Charlotte, NC 39:03 4474  no
Barack Obama August 28, 2008 Denver, CO 41:40 4652  yes
John F. Kerry July 29, 2004 Boston, MA 45:26 5166  yes
Albert Gore, Jr. August 17, 2000 Los Angeles, CA 50:20 5755  yes
William J. Clinton Augu

In [3]:
url = "http://www.presidency.ucsb.edu/platforms.php"
driver.get(url)
a = driver.find_elements_by_xpath("//tr")
year = 2016
rows = list()
for item in a[55:199]:
    
    if len(item.text.split(" ")) > 1:
        arr = item.text.split(" ")
        
        arr = list(filter(None, arr))
        #print(arr)
        if "--" in arr:
            arr.remove("--")
        try:
            year = int(arr[0])
            arr.remove(arr[0])
        except:
            year=year
        if "pdf" in arr:
            arr.remove("pdf")
        word_count = arr[-1]
        arr = arr[:-1]
        if arr[0] == "States" or arr[0] == "Constitutional":
            party=" ".join(arr[0:2])
            arr = arr[2:]
        else:
            party = arr[0]
            arr = arr[1:]
        try:
            electoral_votes = int(arr[-1])
            arr = arr[:-1]
        except:
            electoral_votes = 0
        name = " ".join(arr)
        final = [name, year, party, electoral_votes]
        rows.append(final)
nominees_df = pd.DataFrame(rows, columns=["Name", "Year", "Party", "Electoral Votes"])

In [255]:
a = ['2016', 'Democratic', 'pdf', 'Hillary', 'Clinton', '26,058']
b = ['Republican', 'pdf', 'Donald', 'Trump', '35,467']

In [258]:
try:
    x = int(b[0])
except:
    x = 5

In [279]:
with open("/Users/danielgilberg/data_science/metis/presidential_speeches/data/nominees.pkl", "wb") as picklefile:
    pickle.dump(nominees_df, picklefile)

In [293]:
hrc_url = "http://www.presidency.ucsb.edu/ws/index.php?pid=118051"
hrc_response = requests.get(hrc_url)
hrc_soup = BeautifulSoup(hrc_response.text, "lxml")
speech_name = hrc_soup.find(attrs={"class": "paperstitle"}).text
year = hrc_soup.find(attrs={"class": "docdate"}).text.split(", ")[1]
speech = list()
speech_parts = hrc_soup.find_all("p")
for div in speech_parts:
    speech.append(div.text)
speech = " ".join(speech)

In [315]:
noms_url = "http://www.presidency.ucsb.edu/nomination.php"
noms_response = requests.get(noms_url)
noms_soup = BeautifulSoup(noms_response.text, "lxml")
links = noms_soup.find_all("a")
nom_links = list()
for link in links:
    href = link.attrs["href"]
    if "pid=" in href:
        nom_links.append(href)
len(nom_links)
    

55

In [340]:
type(nom_links)

list

In [348]:
arr = list()
rows = list()
for index, link in enumerate(nom_links):
    response = requests.get(link)
    soup = BeautifulSoup(response.text, "lxml")
    speech_name = soup.find(attrs={"class": "paperstitle"}).text
    if index<24:
        party = "Democratic"
    else:
        party = "Republican"
    year = soup.find(attrs={"class": "docdate"}).text.split(", ")[1]
    speech = list()
    speech_parts = soup.find_all("p")
    for div in speech_parts:
        speech.append(div.text)
    speech = " ".join(speech)
    row = [year, party, speech_name, speech]
    rows.append(row)
#     if "Democratic" not in speech_name and "Republican" not in speech_name:
#         print(link)

        

In [349]:
convention_df = pd.DataFrame(rows, columns=["Year", "Party", "Speech Name", "Speech"])

Counter(arr)

In [350]:
convention_df

,Year,Party,Speech Name,Speech
0,2016,Democratic,Address Accepting the Presidential Nomination ...,Thank you all for the great convention that we...
1,2012,Democratic,693 - Remarks Accepting the Presidential Nomin...,The President. Thank you. Thank you. Thank you...
2,2008,Democratic,Address Accepting the Presidential Nomination ...,"With profound gratitude and great humility, I ..."
3,2004,Democratic,Address Accepting the Presidential Nomination ...,We are here tonight because we love our countr...
4,2000,Democratic,Address Accepting the Presidential Nomination ...,Thank you. Thank you very much. Thank you. Tha...
5,1996,Democratic,Remarks Accepting the Presidential Nomination ...,So many have contributed to the record we have...
6,1992,Democratic,Address Accepting the Presidential Nomination ...,He said he came here tonight because he always...
7,1988,Democratic,Address Accepting the Presidential Nomination ...,A few months ago when Olympia Dukakis in front...
8,1984,Democratic,Address Accepting the Presidential Nomination ...,"My fellow Democrats, my fellow Americans: I ac..."
9,1980,Democratic,Remarks Accepting the Presidential Nomination ...,I thank you for the nomination you've offered ...


In [351]:
with open("/Users/danielgilberg/data_science/metis/presidential_speeches/data/convention_speeches.pkl", "wb") as picklefile:
    pickle.dump(convention_df, picklefile)

In [7]:
nominees_df.at[0, "Electoral Votes"] = 227
nominees_df.at[1, "Electoral Votes"] = 304
with open("/Users/danielgilberg/data_science/metis/presidential_speeches/data/nominees.pkl", "wb") as picklefile:
    pickle.dump(nominees_df, picklefile)

In [40]:
debate_url = "http://www.presidency.ucsb.edu/ws/index.php?pid=6517"
debate_response = requests.get(debate_url)
debate_soup = BeautifulSoup(debate_response.text, "lxml")
arr = debate_soup.find_all("p")
final = list()
for tag in arr:
    final.append(tag.text)
final
final = " ".join(final)
sayings = re.split('[A-Z]{5,}', final)
participants = re.findall('[A-Z]{5,}', final)
participants.insert(0, "Moderator")
list(zip(participants, sayings))
rows = [participants, sayings]
debate = pd.DataFrame(list(zip(participants,sayings)), columns=["Speaker", "Transcript"])
with open("/Users/danielgilberg/data_science/metis/presidential_speeches/data/ford_carter_one.pkl", "wb") as picklefile:
    pickle.dump(debate, picklefile)

In [107]:
miller_center = "https://millercenter.org/the-presidency/presidential-speeches"
driver.get(miller_center)
SCROLL_PAUSE_TIME = 5

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    print(new_height)
    if new_height == last_height:
        break
    last_height = new_height

2866
3754
4642
5530
6418
7306
8194
9082
9970
10858
11746
12634
13522
14410
15298
16186
17074
17962
18850
19738
20626
21514
22402
23290
24178
25066
25954
26842
27730
28618
29506
30394
31282
32170
33058
33946
34834
35721
36609
37497
38385
39273
40161
41049
41937
42825
43713
44601
45489
46377
47265
48153
49041
49929
50817
51705
52593
53481
54369
55257
56145
57033
57921
58809
59697
60585
61473
62361
63249
64137
65025
65913
66801
67689
68577
69465
70353
71241
72129
73059
73503
73503


In [110]:
links = driver.find_elements_by_tag_name("a")
potus_speeches = list()
for link in links:
    if not pd.isnull(link.get_attribute("href")):
        if "speeches/" in link.get_attribute("href"):
            potus_speeches.append(link.get_attribute("href"))

In [114]:
potus_speeches
potus_speeches_df = pd.DataFrame(potus_speeches, columns=["Link"])
with open("/Users/danielgilberg/data_science/metis/presidential_speeches/data/speech_links.pkl", "wb") as picklefile:
    pickle.dump(potus_speeches_df, picklefile)

In [140]:
temp_speech = potus_speeches[20]
driver.get(temp_speech)
button = driver.find_element_by_class_name("expandable-text-trigger")
buttons = driver.find_elements_by_xpath("//*[contains(text(), 'View Transcript')]")
buttons

[<selenium.webdriver.remote.webelement.WebElement (session="7a13d0bdd8ea9692611aaef0f1945983", element="0.7014849375130128-1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="7a13d0bdd8ea9692611aaef0f1945983", element="0.7014849375130128-2")>]

In [199]:
bs = BeautifulSoup(requests.get(temp_speech).text, "lxml")
transcript = bs.find(attrs={"class": "transcript-inner"})
parts = transcript.find_all("p")
speech = " "
arr = []
# for tag in parts:
#     speech.append(tag.text)
# " ".join(speech)
for tag in parts:
    arr.append(unidecode.unidecode(tag.text))
    speech += unidecode.unidecode(tag.text)
president = bs.find(attrs={"class": "president-name"}).text
speech_date = bs.find(attrs={"class": "episode-date"}).text

description = bs.find(attrs={"class": "about-sidebar--intro"}).text


'December 16, 2012'

In [274]:
rows=[]
for index, speech in enumerate(potus_speeches):
    print(index)
    bs = BeautifulSoup(requests.get(speech).text, "lxml")

    transcript = bs.find(attrs={"class": "transcript-inner"})
    transcript1 = bs.find(attrs={"class": "view-transcript"})
    transcript2 =  bs.find(attrs={"class": "about-sidebar--intro"})
    #print(transcript)
    if transcript is not None:
    
        parts = transcript.find_all("p")
        speech = " "
        for tag in parts:
            speech += unidecode.unidecode(tag.text)
    elif transcript1 is not None:
        arr = []
        speech = " "
        parts = transcript1.find_all("p")
        for tag in parts:
            speech +=unidecode.unidecode(tag.text) 
    else: 
        speech = transcript2.find("p")

   
    president = bs.find(attrs={"class": "president-name"}).text
    speech_date = bs.find(attrs={"class": "episode-date"}).text
    print (president, speech_date)
    #description = bs.find(attrs={"class": "about-sidebar--intro"}).text
    row = [president, speech_date, speech]
    rows.append(row)
    
temp_df = pd.DataFrame(rows, columns=["President", "Date",  "Speech"])
temp_df

0
Donald Trump February 28, 2017
1
Donald Trump January 20, 2017
2
Barack Obama May 15, 2016
3
Barack Obama March 22, 2016
4
Barack Obama January 12, 2016
5
Barack Obama June 26, 2015
6
Barack Obama March 07, 2015
7
Barack Obama January 20, 2015
8
Barack Obama November 20, 2014
9
Barack Obama January 28, 2014
10
Barack Obama December 04, 2013
11
Barack Obama September 10, 2013
12
Barack Obama July 24, 2013
13
Barack Obama July 19, 2013
14
Barack Obama April 08, 2013
15
Barack Obama March 21, 2013
16
Barack Obama March 01, 2013
17
Barack Obama February 13, 2013
18
Barack Obama January 29, 2013
19
Barack Obama January 21, 2013
20
Barack Obama December 16, 2012
21
Barack Obama November 06, 2012
22
Barack Obama September 06, 2012
23
Barack Obama January 24, 2012
24
Barack Obama October 21, 2011
25
Barack Obama September 08, 2011
26
Barack Obama June 22, 2011
27
Barack Obama May 25, 2011
28
Barack Obama May 19, 2011
29
Barack Obama May 01, 2011
30
Barack Obama January 25, 2011
31
Barack Oba

,President,Date,Speech
0,Donald Trump,"February 28, 2017","Mr. Speaker, Mr. Vice President, Members of C..."
1,Donald Trump,"January 20, 2017","Chief Justice Roberts, President Carter, Pres..."
2,Barack Obama,"May 15, 2016",Hello Rutgers! (Applause.) R-U rah-rah! (A...
3,Barack Obama,"March 22, 2016",Thank you. (Applause.) Muchas gracias. Tha...
4,Barack Obama,"January 12, 2016","Mr. Speaker, Mr. Vice President, Members of C..."
5,Barack Obama,"June 26, 2015",THE PRESIDENT: Giving all praise and honor t...
6,Barack Obama,"March 07, 2015","UDIENCE MEMBER: We love you, President Obama..."
7,Barack Obama,"January 20, 2015","Mr. Speaker, Mr. Vice President, Members of C..."
8,Barack Obama,"November 20, 2014","My fellow Americans, tonight, I'd like to tal..."
9,Barack Obama,"January 28, 2014","Mr. Speaker, Mr. Vice President, Members of C..."


,President,Speech
0,Abraham Lincoln,15
1,Andrew Jackson,26
2,Andrew Johnson,31
3,Barack Obama,50
4,Benjamin Harrison,23
5,Bill Clinton,39
6,Calvin Coolidge,12
7,Chester A. Arthur,11
8,Donald Trump,2
9,Dwight D. Eisenhower,6


In [229]:
temp_speech = potus_speeches[54]
temp_bs = BeautifulSoup(requests.get(temp_speech).text, "lxml")
paras =temp_bs.find(attrs={"class": "view-transcript"}).find_all("p")
arr = []
for tag in paras:
    arr.append(unidecode.unidecode(tag.text))
print(" ".join(arr))

If there is anyone out there who still doubts that America is a place where all things are possible; who still wonders if the dream of our founders is alive in our time; who still questions the power of our democracy, tonight is your answer.

It's the answer told by lines that stretched around schools and churches in numbers this nation has never seen; by people who waited three hours and four hours, many for the very first time in their lives, because they believed that this time must be different; that their voice could be that difference.

It's the answer spoken by young and old, rich and poor, Democrat and Republican, black, white, Latino, Asian, Native American, gay, straight, disabled and not disabled--Americans who sent a message to the world that we have never been a collection of Red States and Blue States: we are, and always will be, the United States of America.

It's the answer that led those who have been told for so long by so many to be cynical, and fearful, and doubtful

In [256]:
x = potus_speeches[54]
bs1 = BeautifulSoup(requests.get(x).text, "lxml")
transcript = bs1.find(attrs={"class": "transcript-inner"})
transcript1 = bs.find(attrs={"class": "view-transcript"})
# if transcript is not None:
#     parts = transcript.find_all("p")
#     speech = " "
#     for tag in parts:
#         speech += unidecode.unidecode(tag.text)
# speech
transcript1.find

<div class="view-transcript"><h3>Transcript</h3><p>If there is anyone out there who still doubts that America is a place where all things are possible; who still wonders if the dream of our founders is alive in our time; who still questions the power of our democracy, tonight is your answer.<br/>

It's the answer told by lines that stretched around schools and churches in numbers this nation has never seen; by people who waited three hours and four hours, many for the very first time in their lives, because they believed that this time must be different; that their voice could be that difference.<br/>

It's the answer spoken by young and old, rich and poor, Democrat and Republican, black, white, Latino, Asian, Native American, gay, straight, disabled and not disabled—Americans who sent a message to the world that we have never been a collection of Red States and Blue States: we are, and always will be, the United States of America.<br/>

It's the answer that led those who have been tol